In [1]:
import numpy as np
import pandas as pd
from mat4py import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn import metrics
from sklearn.utils import resample

import tensorflow
from tensorflow.keras.callbacks import ModelCheckpoint
import sklearn
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization,Concatenate,concatenate, Input
from tensorflow.keras.layers import Conv2D, Conv1D, MaxPooling1D
#from tensorflow.keras.utils import np_utils
#from tensorflow.keras.utils.vis_utils import model_to_dot
from tensorflow.keras.regularizers import l2

In [2]:
tensorflow.keras.backend.clear_session()

In [3]:
model_ecg = Sequential()
model_ecg.add(BatchNormalization(input_shape=(1408,1)))
model_ecg.add(Conv1D(3, kernel_size=(100),strides=2))
model_ecg.add(Activation("relu"))
model_ecg.add(MaxPooling1D(pool_size=(2),strides=2))
model_ecg.add(Conv1D(50, (10)))
model_ecg.add(MaxPooling1D(pool_size=(2),strides=2))
model_ecg.add(Activation("relu"))
model_ecg.add(Conv1D(30, (30)))
model_ecg.add(MaxPooling1D(pool_size=(2)))
model_ecg.add(Activation("relu"))
model_ecg.add(BatchNormalization())
model_ecg.add(Flatten())
model_ecg.add(Dropout(0.25))
model_ecg.add(Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='softmax'))

model_ecg.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 1408, 1)           4         
_________________________________________________________________
conv1d (Conv1D)              (None, 655, 3)            303       
_________________________________________________________________
activation (Activation)      (None, 655, 3)            0         
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 327, 3)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 318, 50)           1550      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 159, 50)           0         
_________________________________________________________________
activation_1 (Activation)    (None, 159, 50)           0

In [4]:
model_ecg.load_weights('./best_weights_ecg_32layer.hdf5')

In [5]:
weight_values=model_ecg.get_weights()

In [6]:
first_input = Input(shape=(1408,1))
first_batch = BatchNormalization(weights=weight_values[0:4],trainable=False)(first_input)
first_conv= Conv1D(3,kernel_size=(100),strides=2,weights=[weight_values[4],weight_values[5]],trainable=False)(first_batch)
first_act=Activation("relu")(first_conv)
first_maxpool= MaxPooling1D(pool_size=(2),strides=2)(first_act)
first_conv2=Conv1D(50,kernel_size=(10),weights=[weight_values[6],weight_values[7]],trainable=False)(first_maxpool)
first_maxpool2=MaxPooling1D(pool_size=(2),strides=2)(first_conv2)
first_act2=Activation("relu")(first_maxpool2)
first_conv3=Conv1D(30,kernel_size=(30),weights=[weight_values[8],weight_values[9]],trainable=False)(first_act2)
first_maxpool3=MaxPooling1D(pool_size=(2))(first_conv3)
first_act3=Activation("relu")(first_maxpool3)
first_batch2 = BatchNormalization(weights=weight_values[10:14],trainable=False)(first_act3)
first_flatten = Flatten()(first_batch2)
first_dropout=Dropout(0.33)(first_flatten)
first_final=Flatten()(first_dropout)

In [7]:
model_spo2 = Sequential()
model_spo2.add(BatchNormalization(input_shape=(88,1)))
model_spo2.add(Conv1D(6, kernel_size=(25),padding='same'))
model_spo2.add(Activation("relu"))
#model_spo2.add(MaxPooling1D(pool_size=(2)))
model_spo2.add(Conv1D(50, (10),padding='same'))
model_spo2.add(MaxPooling1D(pool_size=(2)))
model_spo2.add(Activation("relu"))
model_spo2.add(Conv1D(30, (15),padding='same'))
model_spo2.add(MaxPooling1D(pool_size=(2)))
model_spo2.add(Activation("relu"))
model_spo2.add(BatchNormalization())
model_spo2.add(Flatten())
model_spo2.add(Dropout(0.25))
model_spo2.add(Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation='softmax'))

model_spo2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_4 (Batch (None, 88, 1)             4         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 88, 6)             156       
_________________________________________________________________
activation_6 (Activation)    (None, 88, 6)             0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 88, 50)            3050      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 44, 50)            0         
_________________________________________________________________
activation_7 (Activation)    (None, 44, 50)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 44, 30)           

In [8]:
model_spo2.load_weights('./best_weights_spo2_3_312layer.hdf5')

In [9]:
weight_spo2=model_spo2.get_weights()

In [10]:
second_input = Input(shape=(88,1))
second_batch = BatchNormalization(weights=weight_spo2[0:4],trainable=False)(second_input)
second_conv= Conv1D(6,kernel_size=(25),padding='same',weights=[weight_spo2[4],weight_spo2[5]],trainable=False)(second_batch)
second_act=Activation("relu")(second_conv)
second_conv2=Conv1D(50,kernel_size=(10),padding='same',weights=[weight_spo2[6],weight_spo2[7]],trainable=False)(second_act)
second_maxpool2=MaxPooling1D(pool_size=(2))(second_conv2)
second_act2=Activation("relu")(second_maxpool2)
second_conv3=Conv1D(30,kernel_size=(15),padding='same',weights=[weight_spo2[8],weight_spo2[9]],trainable=False)(second_maxpool2)
second_maxpool3=MaxPooling1D(pool_size=(2))(second_conv3)
second_act3=Activation("relu")(second_maxpool3)
second_batch2 = BatchNormalization(weights=weight_spo2[10:14],trainable=False)(second_act3)
second_final = Flatten()(second_batch2)


In [11]:
merge_layer=concatenate([first_final, second_final])

In [12]:
#''./best_weights_fusion_12/12.hdf5'
merge_batch1 = BatchNormalization()(merge_layer)
merge_drop2 = Dropout(0.25)(merge_batch1)
merge_dense2 = Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(merge_drop2)
merge_output= Activation("softmax")(merge_dense2)

'''#''./best_weights_fusion_10_12.hdf5'
merge_batch1 = BatchNormalization()(merge_layer)
merge_dense1 = Dense(750)(merge_batch1)
merge_drop2 = Dropout(0.25)(merge_dense1)
merge_dense2 = Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(merge_drop2)
merge_output= Activation("softmax")(merge_dense2)'''

'#\'\'./best_weights_fusion_10_12.hdf5\'\nmerge_batch1 = BatchNormalization()(merge_layer)\nmerge_dense1 = Dense(750)(merge_batch1)\nmerge_drop2 = Dropout(0.25)(merge_dense1)\nmerge_dense2 = Dense(2, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(merge_drop2)\nmerge_output= Activation("softmax")(merge_dense2)'

In [13]:
model_merge = Model(inputs=[first_input, second_input], outputs=merge_output)

In [14]:
model_merge.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1408, 1)]    0                                            
__________________________________________________________________________________________________
batch_normalization_2 (BatchNor (None, 1408, 1)      4           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 655, 3)       303         batch_normalization_2[0][0]      
__________________________________________________________________________________________________
activation_3 (Activation)       (None, 655, 3)       0           conv1d_3[0][0]                   
______________________________________________________________________________________________

In [15]:
batch_size = 64
epochs = 500

opt = tensorflow.keras.optimizers.Adam(learning_rate=0.0001)
model_merge.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [16]:
list_string=['ucddb002','ucddb003','ucddb005','ucddb006','ucddb007','ucddb009',\
             'ucddb010','ucddb012','ucddb014','ucddb015','ucddb017',\
             'ucddb019','ucddb020','ucddb021','ucddb022','ucddb023','ucddb024',\
             'ucddb025','ucddb026','ucddb027','ucddb028']#'ucddb008','ucddb011','ucddb013','ucddb014',



valid_features1=np.zeros((1,1408))
valid_features2=np.zeros((1,1408))
test_features1=np.zeros((1,1408))
test_features2=np.zeros((1,1408))
valid_labels=np.zeros((1,1))
test_labels=np.zeros((1,1))
for l in list_string:
        
        
        spo2_valid = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_spo2_valid.mat')
        spo2_valid = np.array(spo2_valid['spo2_valid'])
        ecg_valid = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_ecg_valid.mat')
        ecg_valid = np.array(ecg_valid['ecg_valid'])
        spo2_valid_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_valid_labels.mat')
        spo2_valid_labels = np.array(spo2_valid_labels['class_valid'])
        valid_features1=np.append(valid_features1,spo2_valid,axis=0)
        valid_features2=np.append(valid_features2,ecg_valid,axis=0)
        valid_labels=np.append(valid_labels,spo2_valid_labels)
       
        
        spo2_test = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_spo2_test.mat')
        spo2_test = np.array(spo2_test['spo2_test'])
        ecg_test = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_ecg_test.mat')
        ecg_test = np.array(ecg_test['ecg_test'])
        spo2_test_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_test_labels.mat')
        spo2_test_labels = np.array(spo2_test_labels['class_test'])
        test_features1=np.append(test_features1,spo2_test,axis=0)
        test_features2=np.append(test_features2,ecg_test,axis=0)
        test_labels=np.append(test_labels,spo2_test_labels)
        
spo2_valid=valid_features1[1:,0::16]
ecg_valid=valid_features2[1:]
valid_labels=valid_labels[1:]
valid_labels = valid_labels.flatten()

spo2_test=test_features1[1:,0::16]
ecg_test=test_features2[1:]
test_labels=test_labels[1:]
test_labels = test_labels.flatten()

In [17]:
train_features1=np.zeros((1,1408))
train_features2=np.zeros((1,1408))

train_labels=np.zeros((1,1))

for l in list_string:
        spo2_train = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_spo2_train.mat')
        spo2_train = np.array(spo2_train['spo2_train'])
        ecg_train = loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_ecg_train.mat')
        ecg_train = np.array(ecg_train['ecg_train'])
        spo2_train_labels=loadmat('D:\\PhD topics\\Datasets\\sleep_apnea\\selected\\'+l+'_train_labels.mat')
        spo2_train_labels = np.array(spo2_train_labels['class_train'])
        train_features1=np.append(train_features1,spo2_train,axis=0)
        train_features2=np.append(train_features2,ecg_train,axis=0)
        train_labels=np.append(train_labels,spo2_train_labels)
        
spo2_train=train_features1[1:,0::16]
ecg_train=train_features2[1:]
train_labels=train_labels[1:]
train_labels = train_labels.flatten()

In [18]:
train_features1=0
train_features2=0
valid_features1=0
valid_features2=0
test_features1=0
test_features2=0

In [19]:
ecg_train_mean=np.mean(ecg_train)
ecg_train_std=np.std(ecg_train)

In [20]:
for i in range(ecg_train.shape[0]):
    ecg_train[i,:]=(ecg_train[i,:]-ecg_train_mean)/ecg_train_std
    
for i in range(ecg_valid.shape[0]):
    ecg_valid[i,:]=(ecg_valid[i,:]-ecg_train_mean)/ecg_train_std
    
for i in range(ecg_test.shape[0]):
    ecg_test[i,:]=(ecg_test[i,:]-ecg_train_mean)/ecg_train_std
    

ecg_train=np.expand_dims(ecg_train, axis=2)
ecg_valid=np.expand_dims(ecg_valid, axis=2)
ecg_test=np.expand_dims(ecg_test, axis=2)

In [21]:
spo2_train_mean=np.mean(spo2_train)
spo2_train_std=np.std(spo2_train)

In [22]:
for i in range(spo2_train.shape[0]):
    spo2_train[i,:]=(spo2_train[i,:]-spo2_train_mean)/spo2_train_std
    
for i in range(spo2_valid.shape[0]):
    spo2_valid[i,:]=(spo2_valid[i,:]-spo2_train_mean)/spo2_train_std
    
for i in range(spo2_test.shape[0]):
    spo2_test[i,:]=(spo2_test[i,:]-spo2_train_mean)/spo2_train_std

spo2_train=np.expand_dims(spo2_train, axis=2)
spo2_valid=np.expand_dims(spo2_valid, axis=2)
spo2_test=np.expand_dims(spo2_test, axis=2)

In [23]:
y_train=train_labels
y_valid=valid_labels
y_test=test_labels

In [24]:
num_classes=2
y_train_encoder = sklearn.preprocessing.LabelEncoder()
y_train_num = y_train_encoder.fit_transform(y_train)
y_train_wide = tensorflow.keras.utils.to_categorical(y_train_num, num_classes)

y_valid_num = y_train_encoder.fit_transform(y_valid)
y_valid_wide = tensorflow.keras.utils.to_categorical(y_valid_num, num_classes)

y_test_num = y_train_encoder.fit_transform(y_test)
y_test_wide = tensorflow.keras.utils.to_categorical(y_test_num, num_classes)

In [26]:
best_weights_filepath_merge = './best_weights_fusion_10_12.hdf5'
mcp_merge = ModelCheckpoint(best_weights_filepath_merge, monitor="val_accuracy",
                      save_best_only=True, save_weights_only=False)
            
history = model_merge.fit([ecg_train,spo2_train], y_train_wide,
         batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=([ecg_valid,spo2_valid], y_valid_wide),
          callbacks=[mcp_merge])

Epoch 1/500
12593/12593 [==============================] - 60s 5ms/step - loss: 0.1060 - accuracy: 0.9740 - val_loss: 0.0652 - val_accuracy: 0.9926
Epoch 2/500
12593/12593 [==============================] - 60s 5ms/step - loss: 0.0717 - accuracy: 0.9850 - val_loss: 0.0629 - val_accuracy: 0.9928
Epoch 3/500
12593/12593 [==============================] - 60s 5ms/step - loss: 0.0688 - accuracy: 0.9854 - val_loss: 0.0594 - val_accuracy: 0.9930
Epoch 4/500
12593/12593 [==============================] - 61s 5ms/step - loss: 0.0674 - accuracy: 0.9852 - val_loss: 0.0609 - val_accuracy: 0.9903
Epoch 5/500
12593/12593 [==============================] - 60s 5ms/step - loss: 0.0653 - accuracy: 0.9855 - val_loss: 0.0599 - val_accuracy: 0.9925
Epoch 6/500
12593/12593 [==============================] - 60s 5ms/step - loss: 0.0642 - accuracy: 0.9853 - val_loss: 0.0573 - val_accuracy: 0.9890
Epoch 7/500
12593/12593 [==============================] - 60s 5ms/step - loss: 0.0633 - accuracy: 0.9852 - val_

12593/12593 [==============================] - 61s 5ms/step - loss: 0.0495 - accuracy: 0.9852 - val_loss: 0.0463 - val_accuracy: 0.9933
Epoch 57/500
12593/12593 [==============================] - 60s 5ms/step - loss: 0.0488 - accuracy: 0.9855 - val_loss: 0.0467 - val_accuracy: 0.9893
Epoch 58/500
12593/12593 [==============================] - 60s 5ms/step - loss: 0.0491 - accuracy: 0.9854 - val_loss: 0.0478 - val_accuracy: 0.9901
Epoch 59/500
12593/12593 [==============================] - 60s 5ms/step - loss: 0.0488 - accuracy: 0.9853 - val_loss: 0.0488 - val_accuracy: 0.9894
Epoch 60/500
12593/12593 [==============================] - 61s 5ms/step - loss: 0.0487 - accuracy: 0.9854 - val_loss: 0.0478 - val_accuracy: 0.9898
Epoch 61/500
12593/12593 [==============================] - 60s 5ms/step - loss: 0.0487 - accuracy: 0.9855 - val_loss: 0.0492 - val_accuracy: 0.9898
Epoch 62/500
12593/12593 [==============================] - 61s 5ms/step - loss: 0.0485 - accuracy: 0.9854 - val_loss: 

Epoch 111/500
12593/12593 [==============================] - 61s 5ms/step - loss: 0.0466 - accuracy: 0.9852 - val_loss: 0.0465 - val_accuracy: 0.9925
Epoch 112/500
12593/12593 [==============================] - 61s 5ms/step - loss: 0.0470 - accuracy: 0.9851 - val_loss: 0.0468 - val_accuracy: 0.9923
Epoch 113/500
12593/12593 [==============================] - 60s 5ms/step - loss: 0.0466 - accuracy: 0.9852 - val_loss: 0.0462 - val_accuracy: 0.9892
Epoch 114/500
12593/12593 [==============================] - 61s 5ms/step - loss: 0.0468 - accuracy: 0.9851 - val_loss: 0.0434 - val_accuracy: 0.9898
Epoch 115/500
12593/12593 [==============================] - 60s 5ms/step - loss: 0.0465 - accuracy: 0.9853 - val_loss: 0.0458 - val_accuracy: 0.9929
Epoch 116/500
12593/12593 [==============================] - 59s 5ms/step - loss: 0.0465 - accuracy: 0.9852 - val_loss: 0.0441 - val_accuracy: 0.9895
Epoch 117/500
12593/12593 [==============================] - 59s 5ms/step - loss: 0.0463 - accuracy:

Epoch 220/500
12593/12593 [==============================] - 61s 5ms/step - loss: 0.0455 - accuracy: 0.9848 - val_loss: 0.0449 - val_accuracy: 0.9894
Epoch 221/500
12593/12593 [==============================] - 61s 5ms/step - loss: 0.0451 - accuracy: 0.9849 - val_loss: 0.0457 - val_accuracy: 0.9896
Epoch 222/500
12593/12593 [==============================] - 61s 5ms/step - loss: 0.0452 - accuracy: 0.9850 - val_loss: 0.0500 - val_accuracy: 0.9893
Epoch 223/500
12593/12593 [==============================] - 59s 5ms/step - loss: 0.0453 - accuracy: 0.9850 - val_loss: 0.0449 - val_accuracy: 0.9931
Epoch 224/500
12593/12593 [==============================] - 59s 5ms/step - loss: 0.0454 - accuracy: 0.9849 - val_loss: 0.0459 - val_accuracy: 0.9896
Epoch 225/500
12593/12593 [==============================] - 58s 5ms/step - loss: 0.0451 - accuracy: 0.9851 - val_loss: 0.0464 - val_accuracy: 0.9929
Epoch 226/500
12593/12593 [==============================] - 59s 5ms/step - loss: 0.0451 - accuracy:

12593/12593 [==============================] - 56s 4ms/step - loss: 0.0445 - accuracy: 0.9849 - val_loss: 0.0469 - val_accuracy: 0.9935
Epoch 329/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0443 - accuracy: 0.9850 - val_loss: 0.0474 - val_accuracy: 0.9890
Epoch 330/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0438 - accuracy: 0.9852 - val_loss: 0.0452 - val_accuracy: 0.9932
Epoch 331/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0447 - accuracy: 0.9849 - val_loss: 0.0478 - val_accuracy: 0.9897
Epoch 332/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0448 - accuracy: 0.9848 - val_loss: 0.0474 - val_accuracy: 0.9886
Epoch 333/500
12593/12593 [==============================] - 57s 4ms/step - loss: 0.0448 - accuracy: 0.9849 - val_loss: 0.0480 - val_accuracy: 0.9899
Epoch 334/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0446 - accuracy: 0.9849 - val_

12593/12593 [==============================] - 57s 5ms/step - loss: 0.0441 - accuracy: 0.9850 - val_loss: 0.0425 - val_accuracy: 0.9924
Epoch 437/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0444 - accuracy: 0.9850 - val_loss: 0.0477 - val_accuracy: 0.9896
Epoch 438/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0440 - accuracy: 0.9850 - val_loss: 0.0421 - val_accuracy: 0.9931
Epoch 439/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0445 - accuracy: 0.9849 - val_loss: 0.0460 - val_accuracy: 0.9891
Epoch 440/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0440 - accuracy: 0.9850 - val_loss: 0.0456 - val_accuracy: 0.9897
Epoch 441/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0445 - accuracy: 0.9847 - val_loss: 0.0430 - val_accuracy: 0.9925
Epoch 442/500
12593/12593 [==============================] - 56s 4ms/step - loss: 0.0441 - accuracy: 0.9849 - val_

In [25]:
model_merge.load_weights('./best_weights_fusion_10_12.hdf5')

In [26]:
y_pred = model_merge.predict([ecg_test,spo2_test])
predict_test=np.argmax(y_pred, axis=1)
predict_test=predict_test.reshape(predict_test.shape[0],1)
cm=confusion_matrix(y_test_num, predict_test)
cm

array([[50205,   163],
       [   35,  1333]], dtype=int64)

In [27]:
(cm[1,1]+cm[0,0])/(cm[1,1]+cm[1,0]+cm[0,0]+cm[0,1])

0.9961728776867171

In [28]:
cm[1,1]/(cm[1,1]+cm[1,0])

0.9744152046783626

In [29]:
cm[0,0]/(cm[0,0]+cm[0,1])

0.9967638182973316